In [6]:
import sys
import os

PROJECT_DIR = '/home/evgenshuben/Desktop/gitReps/YandexCup/'
sys.path.append(PROJECT_DIR)
os.chdir(PROJECT_DIR)

In [61]:
from omegaconf import DictConfig, OmegaConf
import hydra
from hydra import initialize, compose, initialize_config_dir
from hydra.utils import instantiate

import torch.nn as nn
import torch
import torch.nn.functional as F
import pandas as pd
import numpy as np
from tqdm import tqdm
import faiss

In [8]:
%load_ext autoreload
%autoreload 2

from scr.train import Trainer
from scr.data.dataset import CoverDatasetClassifier

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# info

Тут попробую взять эмбеддинги сильной модели на трейне, чтобы попытаться обучить на ближайших кандидатах реранкер

реранкер будет трансформером с ближайшим свяхыванием, ну либо хотя бы просто бустинг на фичах со скорами модели

In [20]:
# Путь к конфигурационному файлу, сохраненному Hydra
config_path = "/home/evgenshuben/Desktop/gitReps/YandexCup/outputs_val/run-6/hydra/.hydra/config.yaml"

cfg = OmegaConf.load(config_path)
# cfg.enviroment.device = 'cpu'


print(cfg.pipeline.model_ckpt)
print(cfg.model)

/home/evgenshuben/Desktop/gitReps/YandexCup/outputs_val/run-6/data/model/best-model-epoch=22-max_secs=50.pt
{'_target_': 'scr.models.backbone.BackBone', 'backbone_name': 'hgnetv2_b4.ssld_stage2_ft_in1k', 'emb_size': 512, 'num_classes': '${num_classes}'}


In [26]:
cfg.dataset.train.dataloader

{'_target_': 'torch.utils.data.DataLoader', 'dataset': '${dataset.train.dataset}', 'batch_size': 64, 'num_workers': 0, 'shuffle': False, 'drop_last': False, 'collate_fn': '${transforms.collate_fn}'}

In [22]:
trainer = Trainer(cfg)

/home/evgenshuben/Desktop/gitReps/YandexCup/scr/train.py:51: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.model.load_state_dict(torch.load(self.cfg.pipeline.model_ckpt

In [52]:
# trainer.model.eval()

In [31]:
for batch in trainer.train_dataloader:
    break

###### dataloader

Нужно сделать инференс модели на трейне для всех эмбеддингов, чтобы в дальнейшем найти ближайших соседей

В целом прям в конфиге гидры поменял параметры - убрал аугментации и убрал из даталодера дроп ласт и шафл

In [51]:
all_embeddings = []
track_ids = [] 

with torch.no_grad():
    for batch in tqdm(trainer.train_dataloader):
        track = batch['anchor'].to(trainer.device)
        track_id = batch['anchor_id']
        
        embeddings = trainer.model(track)['emb']
        all_embeddings.append(embeddings.cpu().numpy())
        track_ids.append(track_id.cpu().numpy())

100%|██████████| 4673/4673 [06:56<00:00, 11.21it/s]


In [55]:
all_embeddings

4673

In [56]:
all_embeddings = np.concatenate(all_embeddings, axis=0)
norms = np.linalg.norm(all_embeddings, axis=1, keepdims=True)
normalized_embeddings = all_embeddings / norms

In [59]:
%%time
np.save(
    '/home/evgenshuben/Desktop/gitReps/YandexCup/outputs_val/run-6/data/train_embeddings.npy', 
    normalized_embeddings
)

CPU times: user 0 ns, sys: 162 ms, total: 162 ms
Wall time: 161 ms


In [60]:
normalized_embeddings = (
    np.load(
        '/home/evgenshuben/Desktop/gitReps/YandexCup/outputs_val/run-6/data/train_embeddings.npy'
    )
)

# nearest neighbours

In [62]:
%%time
d = normalized_embeddings.shape[1]  # размерность эмбеддингов
n_data = normalized_embeddings.shape[0]  # количество векторов в базе данных
n_clusters = 500  # количество кластеров для IVF индекса

# Создание индекса с инвертированным файлом (IVF) и точным поиском (Flat) внутри кластеров
index = faiss.IndexIVFFlat(faiss.IndexFlatL2(d), d, n_clusters, faiss.METRIC_INNER_PRODUCT)



# Обучение индекса (нужно только для IVF индексов)
index.train(normalized_embeddings)

# Добавляем векторы в индекс
index.add(normalized_embeddings)

# Переключаем индекс в режим поиска приближенных ближайших соседей
index.nprobe = 100

CPU times: user 19.6 s, sys: 300 ms, total: 19.9 s
Wall time: 2.37 s


In [63]:
# Предположим, embeddings - это массив эмбеддингов
num_embeddings = len(normalized_embeddings)

# Размер батча
batch_size = 1000
k_neighbors = 200

# Предварительное выделение памяти для хранения всех расстояний и индексов
all_distances = np.empty((num_embeddings, k_neighbors), dtype=np.float32)
all_indices = np.empty((num_embeddings, k_neighbors), dtype=np.int64)

# Проходим по эмбеддингам батчами
for i in tqdm(range(0, num_embeddings, batch_size)):
    # Выбираем батч эмбеддингов
    batch_embeddings = normalized_embeddings[i:i + batch_size]
    
    # Поиск ближайших соседей для батча
    distances, indices = index.search(batch_embeddings, k_neighbors)
    
    # Сохраняем последние Last_neighbors расстояний и индексов в предварительно выделенные массивы
    all_distances[i:i + batch_embeddings.shape[0], :] = distances
    all_indices[i:i + batch_embeddings.shape[0], :] = indices

# Теперь all_distances и all_indices содержат все результаты

100%|██████████| 300/300 [17:39<00:00,  3.53s/it]


In [65]:
all_distances.shape

(299054, 200)

In [67]:
all_indices

array([[     0, 211821, 298660, ..., 130674,  82995, 160914],
       [     1,     10,      5, ..., 100875,  70190, 298160],
       [     2,      7, 229095, ..., 231743, 282913, 186521],
       ...,
       [299051, 237947, 205046, ..., 217533, 220529, 209906],
       [299052, 299053, 154009, ..., 156581, 110842, 137973],
       [299053, 299052, 172246, ..., 272261, 166923, 211230]])

In [66]:
all_distances

array([[1.        , 0.5446053 , 0.538137  , ..., 0.36038807, 0.36023456,
        0.359958  ],
       [1.        , 0.64484715, 0.5379874 , ..., 0.30286157, 0.30279297,
        0.30259752],
       [1.        , 0.5993472 , 0.5541804 , ..., 0.35421175, 0.35386443,
        0.35373074],
       ...,
       [1.0000001 , 0.49141178, 0.44849968, ..., 0.33143944, 0.33137733,
        0.33129835],
       [1.        , 0.5367408 , 0.5084946 , ..., 0.34609652, 0.3460042 ,
        0.3459977 ],
       [0.99999994, 0.5367408 , 0.50835925, ..., 0.3356787 , 0.33563325,
        0.33532375]], dtype=float32)

In [68]:
nearest_track_idx_pd = pd.DataFrame({
    'track_id': np.arange(num_embeddings),  # Индексы треков
    'neighbors': all_indices.tolist()       # Преобразуем индексы соседей в списки
})

In [69]:
nearest_track_idx_pd

,track_id,neighbors
0,0,"[0, 211821, 298660, 251253, 100247, 201287, 20..."
1,1,"[1, 10, 5, 0, 140427, 152554, 207231, 207223, ..."
2,2,"[2, 7, 229095, 6, 209567, 3, 8, 257662, 265336..."
3,3,"[3, 8, 257662, 7, 94376, 274726, 152613, 25766..."
4,4,"[4, 144178, 248566, 168991, 8381, 296969, 2718..."
...,...,...
299049,299049,"[299049, 268646, 299048, 189203, 213919, 27444..."
299050,299050,"[299050, 229088, 214484, 203110, 88629, 88637,..."
299051,299051,"[299051, 237947, 205046, 298370, 205047, 68064..."
299052,299052,"[299052, 299053, 154009, 136319, 227134, 27304..."


In [70]:
track_id_series = pd.Series(trainer.train_dataloader.dataset.track_ids)

In [75]:
# 1. Заменяем индексы в колонке 'track_id' на настоящие трек ID
nearest_track_idx_pd['track_id'] = nearest_track_idx_pd['track_id'].map(track_id_series)
# 2. Заменяем индексы в колонке 'neighbors' на настоящие трек ID
nearest_track_idx_pd['neighbors'] = nearest_track_idx_pd['neighbors'].apply(lambda x: track_id_series[x].tolist())

In [76]:
nearest_track_idx_pd

,track_id,neighbors
0,343223,"[343223, 314991, 254137, 200291, 101812, 12627..."
1,361210,"[361210, 101132, 307473, 343223, 281680, 33874..."
2,114472,"[114472, 247079, 109255, 216454, 192349, 13474..."
3,134744,"[134744, 215467, 258956, 247079, 179511, 30816..."
4,271362,"[271362, 315298, 144962, 332689, 180183, 19886..."
...,...,...
299049,73797,"[73797, 208025, 71481, 247358, 361299, 198107,..."
299050,71855,"[71855, 189726, 361640, 218133, 277305, 203412..."
299051,74678,"[74678, 115338, 146468, 90655, 321988, 102338,..."
299052,72629,"[72629, 75917, 227784, 256769, 190442, 196198,..."


In [78]:
track_to_versions = {}

for clique_id, row in trainer.train_dataloader.dataset.versions.iterrows():
    for track in row['versions']:
        track_to_versions[track] = row['versions']  # Все треки в версии этой клики

In [80]:
def filter_neighbors(track_id, neighbors):
    # Получаем список ремиксов для текущего track_id
    versions = track_to_versions.get(track_id, [])
    # Удаляем соседей, которые являются версиями
    return [neighbor for neighbor in neighbors if neighbor not in versions]

In [82]:
nearest_track_idx_pd['negatives'] = nearest_track_idx_pd.apply(
    lambda row: filter_neighbors(row['track_id'], row['neighbors']), axis=1
)

In [87]:
(200 - nearest_track_idx_pd['negatives'].str.len()).mean()

22.555227483999545

In [88]:
nearest_track_idx_pd

,track_id,neighbors,negatives
0,343223,"[343223, 314991, 254137, 200291, 101812, 12627...","[314991, 254137, 200291, 101812, 126276, 35797..."
1,361210,"[361210, 101132, 307473, 343223, 281680, 33874...","[281680, 338741, 204228, 182103, 189128, 23351..."
2,114472,"[114472, 247079, 109255, 216454, 192349, 13474...","[109255, 192349, 258956, 236469, 221151, 22684..."
3,134744,"[134744, 215467, 258956, 247079, 179511, 30816...","[258956, 179511, 308169, 146032, 136942, 29370..."
4,271362,"[271362, 315298, 144962, 332689, 180183, 19886...","[315298, 144962, 332689, 180183, 198867, 29852..."
...,...,...,...
299049,73797,"[73797, 208025, 71481, 247358, 361299, 198107,...","[208025, 247358, 361299, 198107, 294779, 16328..."
299050,71855,"[71855, 189726, 361640, 218133, 277305, 203412...","[189726, 361640, 218133, 277305, 203412, 15904..."
299051,74678,"[74678, 115338, 146468, 90655, 321988, 102338,...","[115338, 146468, 90655, 321988, 102338, 308750..."
299052,72629,"[72629, 75917, 227784, 256769, 190442, 196198,...","[227784, 256769, 190442, 196198, 205959, 15852..."


In [89]:
def find_positives(row):
    neighbors_set = set(row['neighbors'])  # Преобразуем в множество для более быстрого поиска
    negatives_set = set(row['negatives'])
    
    # Оставляем только те элементы из neighbors, которые не входят в negatives
    positives = list(neighbors_set - negatives_set)
    return positives

In [90]:
nearest_track_idx_pd['positives'] = nearest_track_idx_pd.apply(find_positives, axis=1)

In [91]:
nearest_track_idx_pd

,track_id,neighbors,negatives,positives
0,343223,"[343223, 314991, 254137, 200291, 101812, 12627...","[314991, 254137, 200291, 101812, 126276, 35797...","[307473, 361210, 101132, 343223]"
1,361210,"[361210, 101132, 307473, 343223, 281680, 33874...","[281680, 338741, 204228, 182103, 189128, 23351...","[59234, 101132, 307473, 343223, 361210]"
2,114472,"[114472, 247079, 109255, 216454, 192349, 13474...","[109255, 192349, 258956, 236469, 221151, 22684...","[59234, 216454, 247079, 114472, 215467, 307473..."
3,134744,"[134744, 215467, 258956, 247079, 179511, 30816...","[258956, 179511, 308169, 146032, 136942, 29370...","[59234, 216454, 247079, 114472, 215467, 101132..."
4,271362,"[271362, 315298, 144962, 332689, 180183, 19886...","[315298, 144962, 332689, 180183, 198867, 29852...",[271362]
...,...,...,...,...
299049,73797,"[73797, 208025, 71481, 247358, 361299, 198107,...","[208025, 247358, 361299, 198107, 294779, 16328...","[71481, 73797]"
299050,71855,"[71855, 189726, 361640, 218133, 277305, 203412...","[189726, 361640, 218133, 277305, 203412, 15904...","[74678, 71855]"
299051,74678,"[74678, 115338, 146468, 90655, 321988, 102338,...","[115338, 146468, 90655, 321988, 102338, 308750...","[74678, 71855]"
299052,72629,"[72629, 75917, 227784, 256769, 190442, 196198,...","[227784, 256769, 190442, 196198, 205959, 15852...","[75917, 72629]"


In [92]:
nearest_track_idx_pd.to_parquet('/home/evgenshuben/Desktop/gitReps/YandexCup/outputs_val/reranker/train_dataset_pd.pq')

### reranker_train dataset 

Теперь сделаем датасет плоским

In [ ]:
nearest_track_idx_pd